## Fosberg Fire Weather Index

This notebook briefly walks through how to calculate the wildfire metric `change in annual median # of days with FFWI greater than 50` from Cal-Adapt: Analytics Engine data. This notebook may be expanded upon for inclusion in cae-notebooks in the future. 

**Note:** FFWI is calculated at hourly timesteps which ultimately comprises 100+ GB of data between warming level data and the historical baseline. This notebook retrieves the data, processes it by calculating the metric, and then exports the intermediary files to perform the rest of the calculation and generation of Cal-CRAI metric data. At present, this must be performed on the Analytics Engine Jupyter Hub. 

**Order of operations**:
1. Read data in
2. Calculate base function (FFWI, SPEI, warm nights, etc.)

### Step 0: Import libraries

In [ ]:
import climakitae as ck
from climakitae.explore import warming_levels 
from climakitae.util.utils import add_dummy_time_to_wl
from climakitae.tools.indices import fosberg_fire_index
from climakitae.core.data_interface import DataParameters
import pandas as pd
import numpy as np
import xarray as xr

sims_hist = [
    'WRF_MPI-ESM1-2-HR_r3i1p1f1',
    'WRF_MIROC6_r1i1p1f1', 
    'WRF_EC-Earth3_r1i1p1f1',
    'WRF_TaiESM1_r1i1p1f1', 
]

### Step 1: Retrieve data
We need to calculate:
- 30 year centered around 2.0degC warming level (SSP3-7.0)
- Historical baseline 1981-2010 (Historical Climate)

In [ ]:
## Step 1a) Historical baseline data (1981-2010)
selections = DataParameters()
selections.area_average = 'No'
selections.timescale = 'hourly'
selections.variable_type = 'Derived Index'
selections.variable = 'Fosberg fire weather index'
selections.area_subset = "states"
selections.cached_area = ["CA"]
selections.scenario_historical = ['Historical Climate']
selections.time_slice = (1981, 2010)
selections.resolution = '9 km' ## 45km for testing on AE hub
hist_ds = selections.retrieve()
hist_ds = hist_ds.sel(simulation = sims_hist)
hist_ds

In [ ]:
# calculate metric
ffwi_threshold = 50
ds_ffwi_h = (hist_ds >= ffwi_threshold).groupby('time.year').sum('time', min_count=1).mean(['year', 'simulation']).squeeze()
ds_ffwi_h

In [ ]:
## export intermediary file -- export takes approximately 1 hour due to data size
ck.export(ds_ffwi_h, 'ffwi_hist_9km', 'NetCDF')

GWL data for hourly, CA, 9km, 4 bias-adjusted simulations is ~85 GB. We can use the "simulation" paramater to specifically select simulations breaking it into a more manageable size. We'll do 1 simulation at a time, export each as a new file, and then in the next phase aggregate together. 

**Note**: This is not the recommended approach for retrieving WL data on the Analytics Engine. 

In [ ]:
# MIROC6 r1i1p1f1 2048-12-16 2.0
# EC-Earth3 r1i1p1f1  2032-02-15 
# MPI-ESM1-2-HR_r3i1p1f1 2045-01-16
# TaiESM r1i1p1f1  2040-08-16 2.0
selections = DataParameters()
selections.area_average = 'No'
selections.timescale = 'hourly'
selections.variable_type = 'Derived Index'
selections.variable = 'Fosberg fire weather index'
selections.area_subset = "states"
selections.cached_area = ["CA"]
selections.simulation = ['TaiESM1'] ## order here matters
selections.scenario_historical = ['Historical Climate']
selections.scenario_ssp = ['SSP 3-7.0 -- Business as Usual']
selections.time_slice = (2025, 2054)
selections.resolution = '9 km' ## 45km for testing on AE hub
hist_ds = selections.retrieve()
hist_ds

In [ ]:
ffwi_threshold = 50
ds_ffwi_f = (hist_ds >= ffwi_threshold).groupby('time.year').sum('time', min_count=1).mean(['year']).squeeze()
ds_ffwi_f

In [ ]:
## export intermediary file -- export takes approximately 10 minutes per model due to data size
ck.export(ds_ffwi_f, 'ffwi_proj_9km_MIROC', "NetCDF")
# ck.export(ds_ffwi_f, 'ffwi_proj_9km_ECEarth3', 'NetCDF')
# ck.export(ds_ffwi_f, 'ffwi_proj_9km_TaiESM', 'NetCDF')
# ck.export(ds_ffwi_f, 'ffwi_proj_9km_MPI', 'NetCDF')

**Export note**: Intermediary files for FFWI are in the 2a_subset/climate_risk/wildfire/exposure folder on AWS. 